In [1]:
import glob
import os
from utils import trs2wav, trs2df, parsedf
from pyannote.core import Annotation, Segment, Timeline
import pandas as pd

2023-06-29 12:30:56.755723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 12:30:57.417970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def map_gender(s):
    # MAP speech labels to gender label
    #
    
    if s == '':
        return 'nonspeech'
    
    labels = list(set([e[0] for e in s.split('+') if e[0] in 'HFI']))
    
    if len(labels) == 0:
        return 'nonspeech'
    
    if len(labels) > 1:
        #print(s)
        return 'undefspeech'
    
    if s[0] == 'I':
        return 'undefspeech'
    
    if s[0] == 'H':
        return 'male'
    
    if s[0] == 'F':
        return 'female'



In [3]:
lrecipe = ['iss', 'p_9', 'p_99', 'p_999', 'p_9999', 'p_99999', 'p_999999', 'p_9999999', 'p_99999999', 'p_999999999']
lrec = []
for f in glob.glob('./upload/william/*.trs') + glob.glob('./upload/cmaertens/*.trs'):
    base = os.path.splitext(os.path.basename(f))[0]
    print(base)
    df = trs2df(f)
    df['gender'] = df.label.map(map_gender)
    df = df[df.gender != 'nonspeech']
    
    M = df[df.gender == 'male'].dur.sum()
    F = df[df.gender == 'female'].dur.sum()
    U = df[df.gender == 'undefspeech'].dur.sum()
    rec = [base, M, F, U]
    
    for recipe in lrecipe:
        iss = pd.read_csv('~/Downloads/segments_viterbi_prob/%s/%s.csv' % (recipe, base), sep='\t')
        iss['dur'] = iss.stop - iss.start
        rec.append(iss.dur[iss.labels == 'male'].sum())
        rec.append(iss.dur[iss.labels == 'female'].sum())
    lrec.append(rec)

columns = ['base', 'M', 'F', 'U']
for i in range(len(lrecipe)):
    columns.append('MR%d' % i)
    columns.append('FR%d' % i)
    
df = pd.DataFrame.from_records(lrec, columns=columns)

radio-RTL-20220702T133218-60
tv-BFT-20220327T162139-60
tv-LCI-20210624T142245-60
radio-MUV-20220505T203547-60
tv-ART-20220130T104726-60
tv-LCI-20210523T161025-60
tv-FR4-20210806T203954-60
radio-FCR-20211012T032010-60
radio-RTL-20220725T193856-60
tv-FR4-20210705T133239-60
tv-C+N-20220817T091809-60
radio-RMC-20210619T191036-60
tv-F24-20210902T235242-60
tv-FR5-20220214T151958-60
radio-FBL-20221223T105429-60
radio-FBL-20210702T150454-60
radio-RMC-20211101T190319-60
radio-FBL-20210418T174018-60
radio-FCR-20211030T225430-60
tv-T5M-20220419T224001-60
tv-FR5-20220306T120828-60
radio-FIF-20210116T222221-60
tv-FR2-20221206T050843-60
tv-T5M-20220122T220844-60
tv-FR4-20210116T005308-60
radio-FIF-20221004T145356-60
tv-TF1-20210331T101331-60
radio-FMU-20220107T154738-60
radio-MUV-20220403T074146-60
radio-FBL-20220808T092729-60
tv-C+N-20210214T060402-60
tv-FR2-20210722T190223-60
tv-ART-20220317T094745-60
tv-FR5-20220214T155103-60
tv-BFT-20211221T020755-60
tv-C+N-20211030T121520-60
tv-C+N-20220702T005

In [4]:
df['chan'] = df.base.map(lambda x: x.split('-')[1])
df['time'] = df.base.map(lambda x: int(x[-9:-7]))
df['media'] = df.base.map(lambda x: x.split('-')[0])
df

,base,M,F,U,MR0,FR0,MR1,FR1,MR2,FR2,...,FR6,MR7,FR7,MR8,FR8,MR9,FR9,chan,time,media
0,radio-RTL-20220702T133218-60,50.055,3.452,0.000,54.52,1.58,45.06,11.04,52.02,4.08,...,5.28,50.82,5.28,50.82,5.28,50.82,5.28,RTL,13,radio
1,tv-BFT-20220327T162139-60,41.920,0.000,0.000,52.10,0.00,50.90,1.20,52.10,0.00,...,0.00,52.10,0.00,52.10,0.00,52.10,0.00,BFT,16,tv
2,tv-LCI-20210624T142245-60,10.636,22.418,0.000,9.90,24.74,9.90,24.74,9.90,24.74,...,24.74,9.90,24.74,9.90,24.74,9.90,24.74,LCI,14,tv
3,radio-MUV-20220505T203547-60,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,MUV,20,radio
4,tv-ART-20220130T104726-60,37.573,0.000,0.000,31.60,0.00,31.60,0.00,31.60,0.00,...,0.00,31.60,0.00,31.60,0.00,31.60,0.00,ART,10,tv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,tv-ART-20210711T093525-60,22.687,12.278,0.000,39.36,2.74,29.64,12.46,27.96,14.14,...,15.34,26.76,15.34,26.76,15.34,26.76,15.34,ART,9,tv
281,radio-FBL-20210808T205100-60,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,FBL,20,radio
282,radio-FIT-20221203T020307-60,33.185,8.273,1.559,32.40,9.32,31.62,10.10,31.62,10.10,...,9.44,32.28,9.44,32.28,9.44,32.28,9.44,FIT,2,radio
283,tv-LCI-20220123T050538-60,14.926,38.692,3.595,24.32,34.42,23.24,35.50,21.80,36.94,...,37.18,21.56,37.18,20.60,38.14,20.60,38.14,LCI,5,tv


In [5]:
df.to_csv('REF_SPEECHTIME.csv', index=False)

In [30]:
for k, sdf in df.groupby(['base']):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    RPF = 100 * RF / (RF+RH)
    
    if (RH + RF) < 30:
        continue
    
    M = sdf['MR0'].sum()
    F = sdf['FR0'].sum()
    ISS = 100 * F / (M+F)

    M = sdf['MR4'].sum()
    F = sdf['FR4'].sum()
    SIM = 100 * F / (M+F)
    
    if abs(ISS-SIM) < 25:
        continue
    
    print(k, RH+RF, RPF, ISS, SIM)


('radio-FBL-20210608T220712-60',) 57.90200000000001 33.805395323132196 6.285072951739621 36.81257014590348
('radio-FIF-20220223T164413-60',) 56.17700000000001 11.173611976431635 25.313878520529357 100.0
('tv-ART-20210711T093525-60',) 34.965 35.115115115115124 6.508313539192403 36.437054631828985
('tv-FR3-20220821T140145-60',) 33.97600000000001 42.945020014127614 14.278799612778315 47.5798644724105
('tv-FR3-20221130T065106-60',) 49.29899999999999 10.586421631270415 19.457676804690355 51.81385122755588
('tv-FR4-20210305T100835-60',) 55.77599999999999 28.869047619047617 99.99999999999999 67.28334956183058
('tv-FR4-20210623T093405-60',) 31.427999999999997 50.0 69.40535633227417 100.0
('tv-M6_-20210326T075818-60',) 37.70099999999999 26.845441765470415 26.84887459807072 52.813504823151106
('tv-TFX-20220818T230837-60',) 34.17099999999999 91.72397647127683 66.2357036300348 100.0


/tmp/ipykernel_23626/2541020333.py:4: RuntimeWarning: invalid value encountered in double_scalars
  RPF = 100 * RF / (RF+RH)


In [18]:
for k, sdf in df.groupby('chan'):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    
    RPF = 100 * RF / (RF+RH)
    print(RH, RF, RPF)
    
    for i in range(10):
        H = sdf['MR%d' % i].sum()
        F = sdf['FR%d' % i].sum()
        PF = 100 * F / (F + H)
        print(k, i, RPF-PF, RPF, PF)

228.36950000000002 79.12150000000001 25.73132221756084
ART 0 1.3841725342626958 25.73132221756084 24.347149683298145
ART 1 -1.8495200982520252 25.73132221756084 27.580842315812866
ART 2 -1.7661775046305195 25.73132221756084 27.49749972219136
ART 3 -2.099547879116571 25.73132221756084 27.830870096677412
ART 4 -2.099547879116571 25.73132221756084 27.830870096677412
ART 5 -2.099547879116571 25.73132221756084 27.830870096677412
ART 6 -2.099547879116571 25.73132221756084 27.830870096677412
ART 7 -2.099547879116571 25.73132221756084 27.830870096677412
ART 8 -2.099547879116571 25.73132221756084 27.830870096677412
ART 9 -2.099547879116571 25.73132221756084 27.830870096677412
289.4505 252.3405 46.57524765084692
BFT 0 1.0930655870063433 46.57524765084692 45.482182063840575
BFT 1 0.28576952215243523 46.57524765084692 46.28947812869448
BFT 2 0.32630321578526633 46.57524765084692 46.24894443506165
BFT 3 0.32630321578526633 46.57524765084692 46.24894443506165
BFT 4 0.32630321578526633 46.57524765084

/tmp/ipykernel_23626/2923373443.py:5: RuntimeWarning: invalid value encountered in double_scalars
  RPF = 100 * RF / (RF+RH)
/tmp/ipykernel_23626/2923373443.py:11: RuntimeWarning: invalid value encountered in double_scalars
  PF = 100 * F / (F + H)


In [8]:
lrec = []
for k, sdf in df.groupby('chan'):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    PH = sdf.MISS.sum()
    PF = sdf.FISS.sum()
    R = 100 * RF / (RF + RH)
    P = 100 * PF / (PF + PH)
    lrec.append((k, len(sdf), 100 * (RH+RF)/(len(sdf) * 60),  R-P, R, P))
dfchan = pd.DataFrame.from_records(lrec, columns=['channel', 'nb_minutes', 'speech_percent', 'REF-PRED', 'Femme%Ref', 'Femme%Pred'])
dfchan.to_csv('errChan.csv', index=False)
dfchan

/tmp/ipykernel_19575/3102516267.py:7: RuntimeWarning: invalid value encountered in double_scalars
  R = 100 * RF / (RF + RH)
/tmp/ipykernel_19575/3102516267.py:8: RuntimeWarning: invalid value encountered in double_scalars
  P = 100 * PF / (PF + PH)


,channel,nb_minutes,speech_percent,REF-PRED,Femme%Ref,Femme%Pred
0,ART,9,56.942778,1.384173e+00,25.731322,24.347150
1,BFT,10,90.298500,1.093066e+00,46.575248,45.482182
2,C+N,20,87.624417,-3.253288e-02,38.996836,39.029369
3,C+_,12,49.673750,1.363061e+00,29.600784,28.237723
4,C25,1,68.248333,6.900205e+00,70.949230,64.049024
5,F24,16,76.987813,-1.966175e+00,39.018013,40.984188
6,FBL,18,33.300000,1.387738e+00,66.112501,64.724763
7,FCR,9,76.318889,-1.960265e+00,9.317144,11.277409
8,FIF,16,93.140417,-3.168886e-01,26.502436,26.819324
9,FIP,8,2.271250,-1.421085e-14,100.000000,100.000000


In [9]:
lrec = []
for k, sdf in df.groupby(['media', 'chan', 'time']):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    PH = sdf.MISS.sum()
    PF = sdf.FISS.sum()
    R = 100 * RF / (RF + RH)
    P = 100 * PF / (PF + PH)
    lrec.append((k, len(sdf), 100 * (RH+RF)/(len(sdf) * 60),  R-P, R, P))
dfchan = pd.DataFrame.from_records(lrec, columns=['channel', 'nb_minutes', 'speech_percent', 'REF-PRED', 'Femme%Ref', 'Femme%Pred'])
dfchan.to_csv('errChanDetail.csv', index=False)
dfchan

/tmp/ipykernel_19575/2707038510.py:7: RuntimeWarning: invalid value encountered in double_scalars
  R = 100 * RF / (RF + RH)
/tmp/ipykernel_19575/2707038510.py:8: RuntimeWarning: invalid value encountered in double_scalars
  P = 100 * PF / (PF + PH)


,channel,nb_minutes,speech_percent,REF-PRED,Femme%Ref,Femme%Pred
0,"(radio, FBL, 1)",2,0.000000,NaN,NaN,NaN
1,"(radio, FBL, 2)",1,0.000000,NaN,NaN,NaN
2,"(radio, FBL, 3)",1,0.000000,NaN,NaN,NaN
3,"(radio, FBL, 4)",1,0.000000,NaN,NaN,NaN
4,"(radio, FBL, 5)",1,91.806667,0.000000,100.000000,100.000000
...,...,...,...,...,...,...
217,"(tv, TF1, 19)",2,45.189167,10.771094,40.406071,29.634976
218,"(tv, TF1, 21)",1,32.425000,-4.688474,70.578258,75.266731
219,"(tv, TF1, 22)",1,77.820000,-5.115000,19.112482,24.227482
220,"(tv, TFX, 5)",1,64.086667,0.000000,0.000000,0.000000


In [46]:
lrec = []
for k, sdf in df[df.media == 'radio'].groupby('time'):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    PH = sdf.MISS.sum()
    PF = sdf.FISS.sum()
    R = 100 * RF / (RF + RH)
    P = 100 * PF / (PF + PH)
    lrec.append((k, R-P, R, P))
dfchan = pd.DataFrame.from_records(lrec, columns=['channel', 'REF-PRED', 'Femme%Ref', 'Femme%Pred'])
dfchan

,channel,REF-PRED,Femme%Ref,Femme%Pred
0,0,0.000000,0.000000,0.000000
1,1,-0.024788,7.344971,7.369759
2,2,0.122782,6.361068,6.238286
3,3,5.575096,9.144538,3.569442
4,4,0.000000,0.000000,0.000000
5,5,-1.131524,53.126296,54.257820
6,6,5.636205,24.399744,18.763539
7,7,-2.279880,17.889856,20.169736
8,8,-1.606392,45.575022,47.181414
9,9,1.084095,5.174245,4.090150


In [48]:
lrec = []
for k, sdf in df[df.media == 'tv'].groupby('time'):
    RH = sdf.M.sum() + .5 * sdf.U.sum()
    RF = sdf.F.sum() + .5 * sdf.U.sum()
    PH = sdf.MISS.sum()
    PF = sdf.FISS.sum()
    R = 100 * RF / (RF + RH)
    P = 100 * PF / (PF + PH)
    lrec.append((k, R-P, R, P))
dfchan = pd.DataFrame.from_records(lrec, columns=['channel', 'REF-PRED', 'Femme%Ref', 'Femme%Pred'])
dfchan

,channel,REF-PRED,Femme%Ref,Femme%Pred
0,0,-0.310108,39.323988,39.634096
1,1,5.465603,41.596931,36.131328
2,2,2.435568,55.260786,52.825219
3,3,1.836546,21.530375,19.693829
4,4,-7.513301,59.182861,66.696162
5,5,3.244035,29.040413,25.796377
6,6,-1.583262,37.905374,39.488636
7,7,0.156651,50.913137,50.756487
8,8,-9.255408,34.340024,43.595433
9,9,-1.065267,25.990566,27.055833
